<a href="https://colab.research.google.com/github/mattiabombieri/NeuralNetwork_Tesi_MB/blob/main/NetworkFittingDifferentNetworksUnivariate_MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import math as mt
import matplotlib.pyplot as plt

2.18.0


In [11]:
from google.colab import drive
drive.mount('/content/drive')

StartingValues=np.load('/content/drive/MyDrive/startingvaluesUniVariate.npy')
print(StartingValues)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[ 0.36999885  0.06088184 -0.27843507 ...  0.05552605  0.27597337
   0.46055534]
 [ 0.41627211 -0.32638021  0.30962175 ...  0.10310086 -0.32621497
  -0.0346609 ]
 [ 0.45123417  0.28628765 -0.34419823 ...  0.17506217 -0.22866537
   0.30101809]
 ...
 [ 0.2657931  -0.06870523 -0.13819409 ...  0.24828656  0.03992342
   0.29094948]
 [-0.12807186  0.42792914  0.27162283 ... -0.05987048 -0.47285047
  -0.04101031]
 [ 0.38745462  0.36525259 -0.33679535 ... -0.19790725 -0.3148698
  -0.31740994]]


In [12]:
Y=np.load('/content/drive/MyDrive/outputsUniVariate.npy')
print(Y)

[0.77322275 0.13960061 0.22053087 ... 0.37118548 0.44038337 0.84606551]


In [13]:
import os
os.getcwd()

'/content'

In [14]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Lambda
from tensorflow.keras.losses import MeanSquaredError
#from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Add
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

In [15]:
#Custom activation function
vector_z=[0.25,0.25,0.25,0.25,0,0,0] #This is the vector z we need to choose in Example 2.12
vector_psi=[0.25,0.25,0.25,0.25,0,0,0] #This is the vector that encodes the function psi in Example 2.12
psi=tf.constant(vector_psi)
z=tf.constant(vector_z)

def custom_fct(x):
    psi_mat=tf.expand_dims(psi, axis=0)
    psi_mat_t=tf.transpose(psi_mat)
    #psi_mat_batch=tf.broadcast_to(psi_mat, [3,1,3])
    tmp=tf.matmul(x,psi_mat_t)
    tmp2=K.maximum(tf.zeros_like(tmp),1-K.exp(-tmp))
    tmp3=tmp2*z
    return tmp3#tf.math.scalar_mul(tmp4, z)

In [16]:
approxDim=7 #This is the approximation dimension, the number of basis functions we consider

models = []
for k in range(1,31,1):
    inputs =  tf.keras.Input(shape=(7,))
    nodes = []
    #After each node is a lambda layer that implements our activation function
    lambda_layers= []
    #Final linear forms that are applied
    linear_form_layers=[]
    for j in range(k):
        node = Dense(approxDim, activation=None,trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      bias_initializer='random_normal',
                      name=str('node')+str(j))
        lamb= Lambda(custom_fct,name=str('activation')+str(j))
        lin= Dense(1, activation=None,trainable=True,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      use_bias=False,
                      name=str('linearForm')+str(j))
        nodes = nodes + [node]
        lambda_layers=lambda_layers + [lamb]
        linear_form_layers=linear_form_layers + [lin]


    all_linear_form_outputs=[]
    for j in range(k):
        output_linearmap=nodes[j](inputs)
        output_activation=lambda_layers[j](output_linearmap)
        output_linear_form=linear_form_layers[j](output_activation)
        all_linear_form_outputs=all_linear_form_outputs+[output_linear_form]


    #print(all_linear_form_outputs)
    add_layer=Add()

    if k>1:
        finalOutput=add_layer(all_linear_form_outputs)
    else:
        finalOutput=all_linear_form_outputs[0]
#print(finalOutput)

    model = Model(inputs=inputs, outputs=finalOutput)
    model.compile(optimizer='adam',loss='mean_squared_error')
    models = models + [model]

In [17]:
import time

In [9]:
for k in range(0,len(models),1):
    tic = time.perf_counter()
    models[k].fit(StartingValues, Y, epochs=50, batch_size=10000)
    toc = time.perf_counter()
    print(f"Calc time: {(toc - tic)/60.0:0.4f} minutes")

Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.1056
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.0664
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0656
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0655
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.0654
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.0654
Epoch 7/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0655
Epoch 8/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0654
Epoch 9/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0653
Epoch 10/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0653
Epoch 11/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0654
Epoch 12/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0653
Epoch 13/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0653
Epoch 14/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0652
Epoch 15/50
1000/1000 ━━━━

KeyboardInterrupt: 

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
import time

# Configura Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitora la perdita sulla validazione
    patience=3,          # Numero di epoche senza miglioramenti prima di fermarsi
    restore_best_weights=True  # Ripristina i pesi della migliore epoca
)

# Loop per allenare i modelli
for k in range(len(models)):  # Non serve scrivere range(0, len(models), 1)
    tic = time.perf_counter()

    models[k].fit(
        StartingValues, Y,
        epochs=50,
        batch_size=10000,
        validation_split=0.2,  # Usa il 20% dei dati per la validazione
        callbacks=[early_stopping]  # Attiva l'early stopping
    )

    toc = time.perf_counter()
    print(f"Calc time: {(toc - tic)/60.0:0.4f} minutes")

Epoch 1/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.1477 - val_loss: 0.1333
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.1331 - val_loss: 0.1332
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.1331 - val_loss: 0.1332
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.1333 - val_loss: 0.1332
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.1331 - val_loss: 0.1332
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.1333 - val_loss: 0.1332
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.1333 - val_loss: 0.1332
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.1332 - val_loss: 0.1332
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.1332 - val_loss: 0.1332
Calc time: 1.2623 minutes
Epoch 1/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - loss: 0.1461 - val_loss: 0.1332
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.1328 - val_loss: 0.1332
Epoch 3/50
800/800 ━━━